In [14]:
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
import json
import librosa
import soundfile as sf
import random
import pandas as pd
from tqdm import tqdm
from audiomentations import Compose, PitchShift, TimeStretch, AddBackgroundNoise, Gain, AddGaussianSNR, ClippingDistortion, Lambda

In [2]:
def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    return audio, sr

def save_audio(file_path, audio, sr):
    sf.write(file_path, audio, sr)

In [3]:
def apply_augmentations(augmentations, file_path, output_dir):
    audio, sr = load_audio(file_path)
    
    for aug_name, aug in augmentations.items():
        file_name = os.path.splitext(file_path)[0].split('\\')[-1]
        if len(file_name.split("_")) >= 4:
            continue
        if os.path.exists(f"{output_dir}/{file_name}_{aug_name}.wav"):
            continue
        augmented_audio = aug(samples=audio, sample_rate=sr)
        save_audio(f"{output_dir}/{file_name}_{aug_name}.wav", augmented_audio, sr)

In [ ]:
training_folder = "E:/Giacomo/Tovanella/orig_segments/train"
other_folder = "E:/Giacomo/Tovanella/orig_segments/altro"
os.makedirs(other_folder, exist_ok=True)
for specie in os.listdir(training_folder):
    for audio in os.listdir(os.path.join(training_folder, specie)):
        os.makedirs(os.path.join(other_folder, specie), exist_ok=True)
        if len(audio.split("_")) > 3:
            os.rename(
                os.path.join(training_folder, specie, audio),
                os.path.join(other_folder, specie, audio)
            ) 

In [26]:
def harmonic_distortion(signal, sample_rate, times=5):
    # Apply harmonic distortion to an audio signal
    for _ in range(times):
        for i in range(len(signal)):
            signal[i] = np.sin(2 * np.pi * signal[i])  # Apply quadratic distortion
    return signal


In [11]:
def random_time_shift(signal, sample_rate):
    # divide the signal in two parts in a random point
    # swap the two parts
    split_point = random.randint(0, len(signal))
    return np.concatenate((signal[split_point:], signal[:split_point]))

In [12]:
def wow_resampling(signal, sample_rate):
    am = 3
    fm = 2
    t = np.arange(len(signal))

    # Compute the varying frequency component
    varying_freq = am * np.sin(2 * np.pi * fm * t / len(signal))

    # Generate the resampled signal
    resampled_signal = np.zeros_like(signal)

    for i in range(len(signal)):
    # Calculate the index to sample from, ensuring it's within bounds
        index = int(i + varying_freq[i])
        if 0 <= index < len(signal):
            resampled_signal[i] = signal[index]
        else:
            resampled_signal[i] = 0  # or handle differently, e.g., with previous value, etc.

    return resampled_signal

In [29]:
augmentations = {
    'wowResampling': Lambda(transform=wow_resampling, p=1.0),
    "noise": AddGaussianSNR(min_snr_db=10, max_snr_db=10, p=1.0),
    'clipping': ClippingDistortion(min_percentile_threshold=5, max_percentile_threshold=5, p=1.0),
    'speed_up': TimeStretch(min_rate=0.15, max_rate=0.15, p=1.0),
    'harmonicDistortion': Lambda(transform=harmonic_distortion, p=1.0),
    'gain': Gain(min_gain_db=10, max_gain_db=10, p=1.0),
    'randTimeShift': Lambda(transform=random_time_shift, p=1.0),
    'pitchShiftA': PitchShift(min_semitones=2, max_semitones=2, p=1.0),
    'pitchShiftB': PitchShift(min_semitones=-2, max_semitones=-2, p=1.0),
}

augm_path = "E:/Giacomo/Tovanella/orig_segments/train"
for specie in os.listdir(augm_path):
    files = os.listdir(os.path.join(augm_path, specie))
    if len(files) > 200:
        continue
    print(f"Augmenting {specie}, {len(files)} audios")
    for i in tqdm(range(len(files))):
        audio = files[i]
        apply_augmentations(augmentations, os.path.join(augm_path, specie, audio), os.path.join(augm_path, specie))

Augmenting Dendrocopos major_Great Spotted Woodpecker, 120 audios


  0%|          | 0/120 [00:00<?, ?it/s]

100%|██████████| 120/120 [00:00<00:00, 1038.99it/s]


In [25]:
training_folder = "E:/Giacomo/Tovanella/orig_segments/train"
species_folders = os.listdir(training_folder)
species_count = { folder: {
    "train": len(os.listdir(os.path.join(training_folder, folder))),
    # "valid": len(os.listdir(os.path.join(valid_folder, folder))),
    # "test": len(os.listdir(os.path.join(test_folder, folder))),
    } for folder in species_folders }

info = pd.DataFrame(species_count).T
info.sort_values("train", ascending=False)

,train
Fringilla coelebs_Common Chaffinch,1807
Regulus regulus_Goldcrest,1750
Wind_,1580
Turdus philomelos_Song Thrush,1546
Sylvia atricapilla_Eurasian Blackcap,1546
Regulus ignicapilla_Common Firecrest,936
Periparus ater_Coal Tit,880
Pecking_,870
Loxia curvirostra_Common Crossbill,770
Troglodytes troglodytes_Eurasian Wren,588
